## 分析
這邊試圖處理的問題是：標註為惡意的log與標註為良性的log在語意(sementic)上是否有所不同?
這邊關係到強化學習的代理Agent能否有效地找到標註為惡意的log(可以分辨出某些惡意行為與其他行為不同)並且標上正確的TTP標籤(找出攻擊的方法)

## 方法
嘗試了兩種類型的方法，純粹的詞意與轉換成Provenance Graph後再進行嵌入轉換

#### 一般路徑以及引入的package設定
##### 動態宣告
個別檔案的路徑名稱將其與對應的檔案路徑變數連結起來

In [26]:
import os
import json
import re
import umap
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# =========================
# Path settings
# =========================
pathSAGA = os.path.join("D:\\", "dataset", "SAGA")
pathCasinoLimit = os.path.join("D:\\", "dataset", "CasinoLimit")
pathOutput=os.path.join(pathSAGA,"result")
pathSAGAmalicious=os.path.join(pathSAGA,"SAGA_malicious.json")
print("SAGA exists:", os.path.exists(pathSAGA))

listSAGAFiles=[]
for dirAPTCampaign in os.listdir(pathSAGA):
    pathAPTCampaign=os.path.join(pathSAGA,dirAPTCampaign)
    if os.path.isdir(pathAPTCampaign) and dirAPTCampaign != "result":
        for auditLog in os.listdir(pathAPTCampaign):
            pathAuditLog=os.path.join(pathAPTCampaign,auditLog)
            #print("current processing:",pathAuditLog)
            #print(auditLog)
            if os.path.isfile(pathAuditLog):
                listSAGAFiles.append("path"+"SAGA"+auditLog.split(".")[0])
                globals()["path"+"SAGA"+auditLog.split(".")[0]]=pathAuditLog
print("SAGA所有有的檔案以及其對應路徑")
for i in listSAGAFiles:
    print(i+":"+globals()[i])


def loadSagaJson(path):
    with open(path, "r", encoding="utf-8") as file:
        l = []
        for i in file:
           l.append(json.loads(i)) 
    return l


SAGA exists: True
SAGA所有有的檔案以及其對應路徑
pathSAGAM1:D:\dataset\SAGA\Composite APT Campaigns Dataset\M1.json
pathSAGAM10:D:\dataset\SAGA\Composite APT Campaigns Dataset\M10.json
pathSAGAM2:D:\dataset\SAGA\Composite APT Campaigns Dataset\M2.json
pathSAGAM3:D:\dataset\SAGA\Composite APT Campaigns Dataset\M3.json
pathSAGAM4:D:\dataset\SAGA\Composite APT Campaigns Dataset\M4.json
pathSAGAM5:D:\dataset\SAGA\Composite APT Campaigns Dataset\M5.json
pathSAGAM6:D:\dataset\SAGA\Composite APT Campaigns Dataset\M6.json
pathSAGAM7:D:\dataset\SAGA\Composite APT Campaigns Dataset\M7.json
pathSAGAM8:D:\dataset\SAGA\Composite APT Campaigns Dataset\M8.json
pathSAGAM9:D:\dataset\SAGA\Composite APT Campaigns Dataset\M9.json
pathSAGAG1:D:\dataset\SAGA\Generated APT Campaigns Dataset\G1.json
pathSAGAG10:D:\dataset\SAGA\Generated APT Campaigns Dataset\G10.json
pathSAGAG11:D:\dataset\SAGA\Generated APT Campaigns Dataset\G11.json
pathSAGAG12:D:\dataset\SAGA\Generated APT Campaigns Dataset\G12.json
pathSAGAG13:D:\dat

In [ ]:
###### 個別測試
sagaLogsG16 = loadSagaJson(pathSAGAG16)
sagaLogsG16[0]['srcNode'].keys()

dict_keys(['UUID', 'Name', 'Image', 'Cmdline', 'Type', 'Pid'])

## 語意分析

In [ ]:
def log_to_embedding_text(log):
    """
    Convert a single audit / graph log to a semi-structured text
    (label must NOT be used here)
    """

    src = log.get("srcNode") or {}
    dst = log.get("dstNode") or {} 
    relation = log.get("relation", "unknown")

    proc_name = src.get("Name", "")
    proc_path = src.get("Image", "")
    action = relation

    dst_type = dst.get("Type", "")
    dst_key = dst.get("Key", "") or dst.get("Name", "")

    # simple abstraction
    if proc_path and (
        "System32" in proc_path or "Program Files" in proc_path
    ):
        proc_category = "system process"
    else:
        proc_category = "user process"

    text = f"""
Process: {proc_name}
Category: {proc_category}
Action: {action}
Target type: {dst_type}
Target description: {dst_key}
""".strip()

    return text
# Visualization (Benign vs TTP)
def PCA(df):
    df["color"] = df["label"].apply(lambda x: "blue" if x == "benign" else "red")

    plt.figure(figsize=(8, 6))

    # Benign
    subset_benign = df[df["label"] == "benign"]
    plt.scatter(
        subset_benign["x"],
        subset_benign["y"],
        color="blue",
        alpha=0.4,
        s=15,
        label="benign")

    # Non-benign (all TTP)
    subset_ttp = df[df["label"] != "benign"]
    plt.scatter(
        subset_ttp["x"],
        subset_ttp["y"],
        color="red",
        alpha=0.6,
        s=20,
        label="TTP")

    plt.legend()
    plt.title("PCA of Audit Log Embeddings (Benign vs TTP)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
def umap(embeddings):
    # =========================
    # UMAP
    # =========================
    print("Running UMAP...")

    umap_model = umap.UMAP(
        n_neighbors=15,
        min_dist=0.1,
        n_components=2,
        metric="cosine",
        random_state=42
    )

    emb_2d_umap = umap_model.fit_transform(embeddings)

    df_umap = pd.DataFrame({
        "x": emb_2d_umap[:, 0],
        "y": emb_2d_umap[:, 1],
        "label": labels
    })

    plt.figure(figsize=(8, 6))

    # benign
    plt.scatter(
        df_umap[df_umap["label"] == "benign"]["x"],
        df_umap[df_umap["label"] == "benign"]["y"],
        color="blue",
        alpha=0.3,
        s=12,
        label="benign")

    # TTP (non-benign)
    plt.scatter(
        df_umap[df_umap["label"] != "benign"]["x"],
        df_umap[df_umap["label"] != "benign"]["y"],
        color="red",
        alpha=0.6,
        s=18,
        label="TTP"
    )

    plt.legend()
    plt.title("UMAP of Audit Log Embeddings (Benign vs TTP)")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.grid(True)
    plt.tight_layout()
def tSNE():
    # =========================
    # t-SNE
    # =========================
    print("Running t-SNE...")

    tsne = TSNE(
        n_components=2,
        perplexity=30,
        learning_rate=200,
        metric="cosine",
        random_state=42
    )

    emb_2d_tsne = tsne.fit_transform(embeddings)

    df_tsne = pd.DataFrame({
        "x": emb_2d_tsne[:, 0],
        "y": emb_2d_tsne[:, 1],
        "label": labels
    })

    plt.figure(figsize=(8, 6))

    # benign
    plt.scatter(
        df_tsne[df_tsne["label"] == "benign"]["x"],
        df_tsne[df_tsne["label"] == "benign"]["y"],
        color="blue",
        alpha=0.3,
        s=12,
        label="benign")

    # TTP (non-benign)
    plt.scatter(
        df_tsne[df_tsne["label"] != "benign"]["x"],
        df_tsne[df_tsne["label"] != "benign"]["y"],
        color="red",
        alpha=0.6,
        s=18,
        label="TTP")

    plt.legend()
    plt.title("t-SNE of Audit Log Embeddings (Benign vs TTP)")
    plt.xlabel("t-SNE-1")
    plt.ylabel("t-SNE-2")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:

saga_logs = load_saga_json(pathSAGAG16)#轉換成embedding的範例
print(log_to_embedding_text(saga_logs[0]))#印出其中一條轉換成embedding的範例
print("Total logs:", len(saga_logs))#印出這份SAGA Audit log總共有多少條log

# Prepare embedding inputs
embedding_texts = []
labels = []   # only used for coloring later

for log in tqdm(saga_logs):
    embedding_text = log_to_embedding_text(log)#將log轉換成全文字格式
    embedding_texts.append(embedding_text)#丟進list方便後續處理
    labels.append(log.get("label", "unknown"))

# =========================
# Embedding
# =========================
print("Embedding logs...")
model = SentenceTransformer("cisco-ai/SecureBERT2.0-biencoder")#載入語意嵌入轉換模型
embeddings = model.encode(embedding_texts, show_progress_bar=True)#開始轉換
embeddings = np.array(embeddings)#丟進np array可以加速處理
# 列出所有標籤(等等所有非benign的會被以不同顏色標註所以要列出來，這邊檢查看是否有其他問題)
df_ttp = df[df["label"] == "TTP"]
print("TTP count:", (df["label"] == "TTP").sum())
print(df["label"].value_counts())


In [ ]:
#以下是針對只針對惡意的Audit轉換成語意嵌入後印出圖表
#顯示了不同的TTP在語意空間中也會有不同的分布特性
# 只保留非 benign
df_ttp = df[df["label"] != "benign"]

print("Number of TTP events:", len(df_ttp))
print("Number of TTP classes:", df_ttp["label"].nunique())

# 取得所有 TTP 類別
ttp_labels = sorted(df_ttp["label"].unique())

# 自動產生顏色（tab20 很適合離散類別）
cmap = cm.get_cmap("tab20", len(ttp_labels))

plt.figure(figsize=(8, 6))

for i, lbl in enumerate(ttp_labels):
    subset = df_ttp[df_ttp["label"] == lbl]
    plt.scatter(
        subset["x"],
        subset["y"],
        color=cmap(i),
        label=lbl,
        alpha=0.7,
        s=20
    )

plt.title("PCA of Audit Log Embeddings (Non-Benign TTPs)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)

# TTP 類別多，legend 放外面
plt.legend(
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    fontsize=8,
    title="TTP Label"
)

plt.tight_layout()
plt.show()

## 統計分析
### 資料集結構分析
只看第一層的話SAGA資料集的結構如下：(共5種)
['srcNode','dstNode','relation','timestamp','label']
#### srcNode所有的鍵值(共6種)
 {'Cmdline', 'Image', 'Name', 'Pid', 'Type', 'UUID'},
#### dstNode所有的鍵值(共12種)
 {'Cmdline','Dstaddress','Image','Key','Name','Path','Pid','Port','Srcaddress','Type','UUID','Value'}
#### 全鍵值不重複統計(共17種)
經過掃描後，可以確認不重複地後，SAGA資料集所有的鍵值如下：
{'Image','Path','dstNode','timestamp','srcNode','Key','Port','Type','relation','Name','Cmdline','Pid','Dstaddress','UUID','label','Srcaddress','Value'}

#### 扁平化鍵值統計(共22種)
經過掃描後，不重複地計算且將所有巢狀迴圈壓縮成扁平化之後統計一詞，這個統計可以避免srcNode和dstNode的欄位混淆
 {'relation','timestamp','label','srcNode.UUID','srcNode.Name','srcNode.Image','srcNode.Cmdline','srcNode.Type','srcNode.Pid','dstNode.UUID','dstNode.Key','dstNode.Type','dstNode.Value','dstNode.Name','dstNode.Srcaddress','dstNode.Dstaddress','dstNode.Port','dstNode.Image','dstNode.Cmdline','dstNode.Pid','dstNode','dstNode.Path'}
 
P.S.這不能直接用於建立RL環境的依據，因為SrcNode與DstNode有相似的結構會導致鍵值可能出現兩次但只被不重複地算到一次
### 各個欄位的內容種類統計
這邊的統計是為了知道把每一筆紀錄塞進某個欄位之後看該欄位裡面有幾種值，這關乎怎麼讓RL模型讀取資料
##### 案例:C1.json
內部統計之後每個欄位的種類數量如下
relation : 27
timestamp : 820
label : 8
srcNode.UUID : 95
srcNode.Name : 32
srcNode.Image : 33
srcNode.Cmdline : 75
srcNode.Type : 1
srcNode.Pid : 95
dstNode.UUID : 30996
dstNode.Key : 25013
dstNode.Type : 4
dstNode.Value : 2711
dstNode.Name : 5915
dstNode.Srcaddress : 6
dstNode.Dstaddress : 49
dstNode.Port : 61
dstNode.Image : 16
dstNode.Cmdline : 20
dstNode.Pid : 33
dstNode : 0
dstNode.Path : 526

統計完之後可以進一步建立映射關係，接著就能確定RL的環境要怎麼把狀態傳給RL模型
目前預計統計出每個檔案中的種類建立成一個csv檔案

In [ ]:
#提取所有鍵值的子功能
def collectUniqueKeys(obj,setKeys)->list:
    if isinstance(obj,dict):
        for key,value in obj.items():
            setKeys.add(key)
            collectUniqueKeys(value,setKeys)
    elif isinstance(obj,list):
        for i in obj:
            collectUniqueKeys(i,setKeys)


In [ ]:
# 統計欄位的程式碼
# 提取唯二巢狀結構的srcNode與dstNode
# 同時也做一次扁平化提取所有鍵值
#print(sagaLogsG16[0].keys())#只算第一層的Key
#這邊要找出SAGA Audit log所有存在的鍵以便於定義
setKeys=set()
setKeysSrcNode=set()
setKeysDstNode=set()
listKeys=[]
for i in listSAGAFiles:
    print(i)
    logs=load_saga_json(globals()[i])
    keys=pd.json_normalize(logs).keys()
    for k in keys:
        if k not in listKeys:
            listKeys.append(k)
    collectUniqueKeys(logs,setKeys)
    for j in logs:
        collectUniqueKeys(j['srcNode'],setKeysSrcNode)
        collectUniqueKeys(j['dstNode'],setKeysDstNode)
    

pathSAGAM1
pathSAGAM10
pathSAGAM2
pathSAGAM3
pathSAGAM4
pathSAGAM5
pathSAGAM6
pathSAGAM7
pathSAGAM8
pathSAGAM9
pathSAGAG1
pathSAGAG10
pathSAGAG11
pathSAGAG12
pathSAGAG13
pathSAGAG14
pathSAGAG15
pathSAGAG16
pathSAGAG17
pathSAGAG18
pathSAGAG19
pathSAGAG2
pathSAGAG20
pathSAGAG3
pathSAGAG4
pathSAGAG5
pathSAGAG6
pathSAGAG7
pathSAGAG8
pathSAGAG9
pathSAGAC1
pathSAGAC2
pathSAGAC3
pathSAGAC4
pathSAGAC5
pathSAGAC6
pathSAGAC7
pathSAGAC8


In [ ]:
#顯示統計欄位的結果
#確認結果(jupyter notebook的好處就是這邊)
setKeys,setKeysSrcNode,setKeysDstNode,listKeys

({'Cmdline',
  'Dstaddress',
  'Image',
  'Key',
  'Name',
  'Path',
  'Pid',
  'Port',
  'Srcaddress',
  'Type',
  'UUID',
  'Value',
  'dstNode',
  'label',
  'relation',
  'srcNode',
  'timestamp'},
 {'Cmdline', 'Image', 'Name', 'Pid', 'Type', 'UUID'},
 {'Cmdline',
  'Dstaddress',
  'Image',
  'Key',
  'Name',
  'Path',
  'Pid',
  'Port',
  'Srcaddress',
  'Type',
  'UUID',
  'Value'},
 ['relation',
  'timestamp',
  'label',
  'srcNode.UUID',
  'srcNode.Name',
  'srcNode.Image',
  'srcNode.Cmdline',
  'srcNode.Type',
  'srcNode.Pid',
  'dstNode.UUID',
  'dstNode.Key',
  'dstNode.Type',
  'dstNode.Value',
  'dstNode.Name',
  'dstNode.Srcaddress',
  'dstNode.Dstaddress',
  'dstNode.Port',
  'dstNode.Image',
  'dstNode.Cmdline',
  'dstNode.Pid',
  'dstNode',
  'dstNode.Path'])

In [16]:

listLogs=[]
with open(pathSAGAC1,"r",encoding='utf-8')as file:
                for line in file:
                    listLogs.append(json.loads(line))
pd.json_normalize(listLogs).head()


,relation,timestamp,label,srcNode.UUID,srcNode.Name,srcNode.Image,srcNode.Cmdline,srcNode.Type,srcNode.Pid,dstNode.UUID,...,dstNode.Value,dstNode.Name,dstNode.Srcaddress,dstNode.Dstaddress,dstNode.Port,dstNode.Image,dstNode.Cmdline,dstNode.Pid,dstNode,dstNode.Path
0,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,300f1c04-bc28-5ea1-9a08-61f9d606465c,...,02 00 00 00 00 00 40 00 02 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,76c7c50d-2d95-5ada-890d-622196ef113a,...,02 00 00 00 00 00 20 00 00 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,d97a5b6b-efa7-56fb-b0c4-800c3d5de336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,d97a5b6b-efa7-56fb-b0c4-800c3d5de336,...,02 00 00 00 FF FF 5F 00 06 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,cb37fa03-3743-5c9d-aedc-618c18a7cf45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

listColumns=['relation','timestamp','label','srcNode.UUID','srcNode.Name','srcNode.Image','srcNode.Cmdline','srcNode.Type','srcNode.Pid','dstNode.UUID','dstNode.Key','dstNode.Type','dstNode.Value','dstNode.Name','dstNode.Srcaddress','dstNode.Dstaddress','dstNode.Port','dstNode.Image','dstNode.Cmdline','dstNode.Pid','dstNode','dstNode.Path']#有甚麼樣的欄位取決於現在讀哪一個資料集
df=pd.DataFrame(pd.json_normalize(listLogs),columns=listColumns)

for i in listColumns:
    print(i,":",df[i].nunique())

,relation,timestamp,label,srcNode.UUID,srcNode.Name,srcNode.Image,srcNode.Cmdline,srcNode.Type,srcNode.Pid,dstNode.UUID,...,dstNode.Value,dstNode.Name,dstNode.Srcaddress,dstNode.Dstaddress,dstNode.Port,dstNode.Image,dstNode.Cmdline,dstNode.Pid,dstNode,dstNode.Path
0,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,300f1c04-bc28-5ea1-9a08-61f9d606465c,...,02 00 00 00 00 00 40 00 02 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,76c7c50d-2d95-5ada-890d-622196ef113a,...,02 00 00 00 00 00 20 00 00 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,d97a5b6b-efa7-56fb-b0c4-800c3d5de336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,d97a5b6b-efa7-56fb-b0c4-800c3d5de336,...,02 00 00 00 FF FF 5F 00 06 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,cb37fa03-3743-5c9d-aedc-618c18a7cf45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
pd.json_normalize(loadSagaJson(globals()[listSAGAFiles[0]])).head()

,relation,timestamp,label,srcNode.UUID,srcNode.Name,srcNode.Image,srcNode.Cmdline,srcNode.Type,srcNode.Pid,dstNode.UUID,...,dstNode.Value,dstNode.Name,dstNode.Srcaddress,dstNode.Dstaddress,dstNode.Port,dstNode.Image,dstNode.Cmdline,dstNode.Pid,dstNode,dstNode.Path
0,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,300f1c04-bc28-5ea1-9a08-61f9d606465c,...,02 00 00 00 00 00 40 00 02 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,76c7c50d-2d95-5ada-890d-622196ef113a,...,02 00 00 00 00 00 20 00 00 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,d97a5b6b-efa7-56fb-b0c4-800c3d5de336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,d97a5b6b-efa7-56fb-b0c4-800c3d5de336,...,02 00 00 00 FF FF 5F 00 06 00 00 00 00 00 00 00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RegQueryValue,1655179745,benign,6376ef5f-def3-56d4-a64a-dc585d30ea4a,SearchFilterHost.exe,C:\Windows\system32\SearchFilterHost.exe,"""C:\Windows\system32\SearchFilterHost.exe"" 0 8...",Process,7736,cb37fa03-3743-5c9d-aedc-618c18a7cf45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
dfStatistical=pd.DataFrame(columns=listColumns)
#這邊我想要做一個csv檔案紀錄各個json檔案裡面各個欄位有哪些種類

for k in range(len(listSAGAFiles)):
    #先讀取檔案                                                             
    logs=loadSagaJson(globals()[listSAGAFiles[k]])
    #再將log換成pd.dataFrame且扁平化
    logsNormalized=pd.json_normalize(logs)
    #載入進有所有欄位的dataFrame
    dfTemp=pd.DataFrame(logsNormalized,columns=listColumns)
    #統計
    dictSAGAFileStatistical={key:dfTemp[key].nunique() if key in dfTemp else 0 for key in listColumns }
    print(dictSAGAFileStatistical)
    dfStatistical=dfStatistical._append({"name":listSAGAFiles[k],**dictSAGAFileStatistical},ignore_index=True)
    
cols = ['name'] + [c for c in dfStatistical.columns if c != 'name']
dfStatistical = dfStatistical[cols]
dfStatistical.to_csv("aaa.csv",index=False)

{'relation': 27, 'timestamp': 834, 'label': 21, 'srcNode.UUID': 108, 'srcNode.Name': 42, 'srcNode.Image': 43, 'srcNode.Cmdline': 88, 'srcNode.Type': 1, 'srcNode.Pid': 108, 'dstNode.UUID': 32103, 'dstNode.Key': 25201, 'dstNode.Type': 4, 'dstNode.Value': 2712, 'dstNode.Name': 6825, 'dstNode.Srcaddress': 8, 'dstNode.Dstaddress': 53, 'dstNode.Port': 66, 'dstNode.Image': 28, 'dstNode.Cmdline': 36, 'dstNode.Pid': 49, 'dstNode': 0, 'dstNode.Path': 1414}
{'relation': 27, 'timestamp': 780, 'label': 21, 'srcNode.UUID': 246, 'srcNode.Name': 55, 'srcNode.Image': 58, 'srcNode.Cmdline': 189, 'srcNode.Type': 1, 'srcNode.Pid': 242, 'dstNode.UUID': 299936, 'dstNode.Key': 65139, 'dstNode.Type': 4, 'dstNode.Value': 8494, 'dstNode.Name': 234539, 'dstNode.Srcaddress': 8, 'dstNode.Dstaddress': 124, 'dstNode.Port': 262, 'dstNode.Image': 41, 'dstNode.Cmdline': 122, 'dstNode.Pid': 168, 'dstNode': 0, 'dstNode.Path': 846}
{'relation': 27, 'timestamp': 664, 'label': 18, 'srcNode.UUID': 67, 'srcNode.Name': 26, 'sr

In [44]:


dfStatistical.to_csv("aaa.csv",index=False)

In [ ]:
dfStatistical=pd.DataFrame(columns=listColumns)
listColumns=['relation','timestamp','label','srcNode.UUID','srcNode.Name','srcNode.Image','srcNode.Cmdline','srcNode.Type','srcNode.Pid','dstNode.UUID','dstNode.Key','dstNode.Type','dstNode.Value','dstNode.Name','dstNode.Srcaddress','dstNode.Dstaddress','dstNode.Port','dstNode.Image','dstNode.Cmdline','dstNode.Pid','dstNode','dstNode.Path']#有甚麼樣的欄位取決於現在讀哪一個資料集
for i in listSAGAFiles:
    print(i+":"+globals()[i])
    listLogs=loadSagaJson(globals()[i])
    pd.json_normalize(listLogs).head()

pathSAGAM1:D:\dataset\SAGA\Composite APT Campaigns Dataset\M1.json
pathSAGAM10:D:\dataset\SAGA\Composite APT Campaigns Dataset\M10.json
pathSAGAM2:D:\dataset\SAGA\Composite APT Campaigns Dataset\M2.json
pathSAGAM3:D:\dataset\SAGA\Composite APT Campaigns Dataset\M3.json
pathSAGAM4:D:\dataset\SAGA\Composite APT Campaigns Dataset\M4.json
pathSAGAM5:D:\dataset\SAGA\Composite APT Campaigns Dataset\M5.json
pathSAGAM6:D:\dataset\SAGA\Composite APT Campaigns Dataset\M6.json
pathSAGAM7:D:\dataset\SAGA\Composite APT Campaigns Dataset\M7.json
pathSAGAM8:D:\dataset\SAGA\Composite APT Campaigns Dataset\M8.json
pathSAGAM9:D:\dataset\SAGA\Composite APT Campaigns Dataset\M9.json
pathSAGAG1:D:\dataset\SAGA\Generated APT Campaigns Dataset\G1.json
pathSAGAG10:D:\dataset\SAGA\Generated APT Campaigns Dataset\G10.json
pathSAGAG11:D:\dataset\SAGA\Generated APT Campaigns Dataset\G11.json
pathSAGAG12:D:\dataset\SAGA\Generated APT Campaigns Dataset\G12.json
pathSAGAG13:D:\dataset\SAGA\Generated APT Campaigns Da